# WBC Segmentation (Unstained) - Kaggle GPU Notebook

Run this end-to-end in a Kaggle Notebook with GPU. Adjust the dataset path as noted below.

In [ ]:
# Clone the repo if not already present (Kaggle starts in /kaggle/working)
import os, subprocess
from pathlib import Path

REPO_URL = "https://github.com/emctalib/dip-blood_cell_segmentations.git"
REPO_DIR = Path("/kaggle/working/dip-blood_cell_segmentations")

if not REPO_DIR.exists():
    subprocess.run(["git", "clone", REPO_URL, str(REPO_DIR)], check=True)
os.chdir(REPO_DIR)
print("CWD:", Path.cwd())

In [ ]:
# Install dependencies
!pip install -r requirements.txt

In [ ]:
# Point to your Kaggle dataset containing data/train|val|test with images + masks (or annotations)
# Example: upload a dataset and set DATASET_BASE to /kaggle/input/your-dataset-name
from pathlib import Path
import os, shutil

DATASET_BASE = Path("/kaggle/input/your-dataset")  # TODO: set to your dataset name
TARGET = Path("data")

if not DATASET_BASE.exists():
    raise FileNotFoundError(f"Dataset path not found: {DATASET_BASE}. Update DATASET_BASE above.")

# If data is already structured as data/train/images etc, just symlink
if TARGET.exists():
    shutil.rmtree(TARGET)
os.symlink(DATASET_BASE, TARGET)
print("Linked", DATASET_BASE, "->", TARGET)

# If masks are not pre-generated but annotations exist, create masks
for split in ["train", "val", "test"]:
    img_dir = TARGET / split / "images"
    ann_dir = TARGET / split / "annotations"
    mask_dir = TARGET / split / "masks"
    if img_dir.exists() and ann_dir.exists() and not mask_dir.exists():
        mask_dir.mkdir(parents=True, exist_ok=True)
        !python prepare_masks.py --images-dir {img_dir} --annotations-dir {ann_dir} --output-dir {mask_dir}
    else:
        print(f"Split {split}: images={img_dir.exists()}, annotations={ann_dir.exists()}, masks={mask_dir.exists()}")

In [ ]:
# Train (baseline config uses AMP and will leverage GPU if available)
!python train.py --config experiments/baseline.yaml

In [ ]:
# Evaluate on val/test and export qualitative masks for the report/deck
!python evaluate.py --config experiments/baseline.yaml --checkpoint outputs/baseline/checkpoints/best.pt --split val --save-dir outputs/preds_val --limit 20
!python evaluate.py --config experiments/baseline.yaml --checkpoint outputs/baseline/checkpoints/best.pt --split test --save-dir outputs/preds_test --limit 20